In [ ]:
# default_exp funcs.output_trafo

In [ ]:
%load_ext autoreload
%autoreload 2

# Transformations of sampled data and model output into emitter dataframes

In [ ]:
#export
from decode_fish.imports import *
from decode_fish.funcs.utils import *
import torch.nn.functional as F
from decode_fish.funcs.plotting import *
from decode_fish.funcs.emitter_io import * 

In [ ]:
#export
# def sample_to_df(locs, x_os, y_os, z_os, ints, px_size_zyx=[100,100,100], channels=16, n_bits=4):
    
#     x = locs[-1] + x_os + 0.5 
#     y = locs[-2] + y_os + 0.5 
#     z = locs[-3] + z_os + 0.5 
    
#     b_inds = torch.cat([torch.tensor([0], device=x_os.device),((x_os[1:] - x_os[:-1]).nonzero() + 1)[:,0], 
#                         torch.tensor([len(x_os)], device=x_os.device)])
#     n_gt = len(b_inds) - 1

#     frame_idx = locs[0]
#     ch_idx = locs[1]
    
#     loc_idx = []
#     for i in range(n_gt):
#         loc_idx += [i] * (b_inds[i+1] - b_inds[i])
    
#     df = DF({'loc_idx': loc_idx,
#              'frame_idx': frame_idx.cpu(),
#              'ch_idx': ch_idx.cpu(),
#              'x': x.cpu()*px_size_zyx[2],
#              'y': y.cpu()*px_size_zyx[1], 
#              'z': z.cpu()*px_size_zyx[0],
#              'int': ints.cpu()}) 
    
#     int_arr = np.zeros([n_gt, channels])
#     int_arr[df['loc_idx'], df['ch_idx']] = ints.cpu()
    
#     df = df.iloc[cpu(b_inds[:-1])]
#     for i in range(16):
#         df[f'int_{i}'] = int_arr[:,i]

#     return df

def sample_to_df(locs, x_os, y_os, z_os, ints, codes, px_size_zyx=[100,100,100], channels=16, n_bits=4):
    
    x = locs[-1] + x_os + 0.5 
    y = locs[-2] + y_os + 0.5 
    z = locs[-3] + z_os + 0.5 
    
    b_inds = torch.cat([torch.tensor([0], device=x_os.device),((x_os[1:] - x_os[:-1]).nonzero() + 1)[:,0], 
                        torch.tensor([len(x_os)], device=x_os.device)])
    n_gt = len(b_inds) - 1

    frame_idx = locs[0]
    ch_idx = locs[1]
    
    loc_idx = []
    for i in range(n_gt):
        loc_idx += [i] * (b_inds[i+1] - b_inds[i])
    
    df = DF({'loc_idx': loc_idx,
             'frame_idx': frame_idx.cpu(),
             'x': x.cpu()*px_size_zyx[2],
             'y': y.cpu()*px_size_zyx[1], 
             'z': z.cpu()*px_size_zyx[0]}) 
    
    int_arr = np.zeros([n_gt, channels])
    int_arr[df['loc_idx'], ch_idx.cpu()] = ints.cpu()
    
    df = df.iloc[cpu(b_inds)[:-1]]
    for i in range(16):
        df[f'int_{i}'] = int_arr[:,i]
        
    df['code_inds'] = codes
    df['ints'] = int_arr.sum(-1)

    return df

def df_to_micro(df, px_size_zyx=[100,100,100]):
    
    locs = tuple([torch.tensor(df['frame_idx'],dtype=torch.int64).cuda(),
                 torch.zeros(len(df),dtype=torch.int64).cuda(),
                 torch.tensor((df['z']/px_size_zyx[0] + 0.5),dtype=torch.int64).cuda(),
                 torch.tensor((df['y']/px_size_zyx[1] + 0.5),dtype=torch.int64).cuda(),
                 torch.tensor((df['x']/px_size_zyx[2] + 0.5),dtype=torch.int64).cuda()])
    z = (torch.tensor(df['z'],dtype=torch.float32).cuda()-locs[2]*px_size_zyx[0])/px_size_zyx[0] - 0.5
    y = (torch.tensor(df['y'],dtype=torch.float32).cuda()-locs[3]*px_size_zyx[1])/px_size_zyx[1] - 0.5
    x = (torch.tensor(df['x'],dtype=torch.float32).cuda()-locs[4]*px_size_zyx[2])/px_size_zyx[2] - 0.5
    ints = torch.tensor(df['int']).cuda()

    return locs, x, y, z, ints

In [ ]:
from decode_fish.funcs.merfish_eval import *
bench_df, code_ref, targets = get_benchmark()
code_inds = np.stack([np.nonzero(c)[0] for c in code_ref])

13832


In [ ]:
from decode_fish.engine.point_process import PointProcessUniform
point_process = PointProcessUniform(local_rate = torch.ones([7,1,1,48,48]).cuda()*.3, int_conc=3, int_rate=1, int_loc=1, sim_iters=1, channels=16, n_bits=4, codebook=torch.tensor(code_inds), int_option=1)
locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape, codes = point_process.sample(from_code_book=True, phasing=False)

/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.7/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU2 NVS 510 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [ ]:
#export
class SIPostProcess(torch.nn.Module):
    
    def __init__(self, m1_threshold:float = 0.03, m2_threshold:float = 0.3, samp_threshold=0.1, px_size_zyx=[100,100,100], diag=0):
        
        super().__init__()
        self.m1_threshold = m1_threshold
        self.m2_threshold = m2_threshold
        self.samp_threshold = samp_threshold
        self.diag = diag
        self.px_size_zyx = px_size_zyx
        
        if not diag:
            d1 = 0; d2 = 0
        else:
            d1 = 1/np.sqrt(2); d2 = 1/np.sqrt(3)
#             d1 = 1; d2 = 1
        self.filt = torch.FloatTensor([[[d2,d1,d2],[d1,1,d1],[d2,d1,d2]],
                                       [[d1, 1,d1],[1, 1, 1],[d1, 1,d1]],
                                       [[d2,d1,d2],[d1,1,d1],[d2,d1,d2]]])[None,None]
        
    def forward(self, logits):

        device = logits.device
        p = torch.sigmoid(logits)

        with torch.no_grad():

            p_copy = p + 0

            # probability values > threshold are regarded as possible locations
            p_clip = torch.where(p>self.m1_threshold,p,torch.zeros_like(p))

            # localize maximum values within a 3x3 patch
            pool = F.max_pool3d(p_clip,3,1,padding=1)
            max_mask1 = torch.eq(p, pool).float()

            # Add probability values from the 4 adjacent pixels
            conv = F.conv3d(p, self.filt.to(device) ,padding=1)
            p_ps1 = (max_mask1 * conv)

            # In order do be able to identify two fluorophores in adjacent pixels we look for probablity values > 0.5 that are not part of the first mask

            p_copy *= (1-max_mask1)
            p_clip = torch.where(p_copy>self.m2_threshold, p_copy,torch.zeros_like(p_copy))
            max_mask2 = torch.where(p_copy>self.m2_threshold, torch.ones_like(p_copy),torch.zeros_like(p_copy))
            p_ps2 = max_mask2*conv

            # This is our final clustered probablity which we then threshold (normally > 0.7) to get our final discrete locations 
            p_ps = p_ps1 + p_ps2        

            return p_ps
        
    def get_si_resdict(self, res_dict, p_si=None):
        
        if p_si is None:
            p_si = self.forward(res_dict['logits'])
            
        res_dict['Probs_si'] = p_si
        res_dict['Samples_si'] = torch.where(res_dict['Probs_si'] > self.samp_threshold, torch.ones_like(res_dict['Probs_si']), torch.zeros_like(res_dict['Probs_si']))
        
        return res_dict
        
    def get_df(self, res_dict, p_si=None, softmax=False):
        
        res_dict = self.get_si_resdict(res_dict, p_si)
        
        res_dict = {k:v.cpu() for (k,v) in res_dict.items()}
        locations = res_dict['Samples_si'].nonzero(as_tuple=True)
        ch0_locs = locations[0], locations[1]*0, locations[2] ,locations[3], locations[4]

        pos_x, pos_y, pos_z = locations[-1] ,locations[-2], locations[-3]
        
        x = pos_x + res_dict['xyzi_mu'][:,[0]][ch0_locs] + 0.5 
        y = pos_y + res_dict['xyzi_mu'][:,[1]][ch0_locs] + 0.5 
        z = pos_z + res_dict['xyzi_mu'][:,[2]][ch0_locs]

        loc_idx = torch.arange(len(x))
        frame_idx = locations[0]

        df = DF({'loc_idx': loc_idx,
                 'frame_idx': frame_idx,
                 'code_inds': locations[1],
                 'x': x*self.px_size_zyx[2],
                 'y': y*self.px_size_zyx[1], 
                 'z': z*self.px_size_zyx[0], 
                 'prob': res_dict['Probs_si'][locations], 
                 'int': res_dict['xyzi_mu'][:,[3]][ch0_locs], 
                 'int_sig': res_dict['xyzi_sigma'][:,[3]][ch0_locs], 
                 'x_sig': res_dict['xyzi_sigma'][:,[0]][ch0_locs]*self.px_size_zyx[0], 
                 'y_sig': res_dict['xyzi_sigma'][:,[1]][ch0_locs]*self.px_size_zyx[1], 
                 'z_sig': res_dict['xyzi_sigma'][:,[2]][ch0_locs]*self.px_size_zyx[2],
                 'comb_sig': torch.sqrt(res_dict['xyzi_sigma'][:,[0]][ch0_locs]**2
                                       +res_dict['xyzi_sigma'][:,[1]][ch0_locs]**2
                                       +res_dict['xyzi_sigma'][:,[2]][ch0_locs]**2)})

        return df
        
    def get_micro_inp(self, res_dict, code_inds, p_si=None, n_bits = 4, channel=0):

        res_dict = self.get_si_resdict(res_dict, p_si)
        locations = res_dict['Samples_si'].nonzero(as_tuple=True)

        xyzi_ix = [locations[0],locations[2],locations[3], locations[4]]
        x_os_3d = res_dict['xyzi_mu'][:,0][xyzi_ix]
        y_os_3d = res_dict['xyzi_mu'][:,1][xyzi_ix]
        z_os_3d = res_dict['xyzi_mu'][:,2][xyzi_ix]
        ints_3d = res_dict['xyzi_mu'][:,3][xyzi_ix]
        # output_shape  = res_dict['Samples_si'].shape
        x_os_3d = x_os_3d.repeat_interleave(n_bits)
        y_os_3d = y_os_3d.repeat_interleave(n_bits)
        z_os_3d = z_os_3d.repeat_interleave(n_bits)
        ints_3d = ints_3d.repeat_interleave(n_bits)/n_bits

        locations = [locations[0].repeat_interleave(n_bits),
                     torch.tensor(code_inds)[locations[1]].reshape(-1),
                     locations[2].repeat_interleave(n_bits),
                     locations[3].repeat_interleave(n_bits), 
                     locations[4].repeat_interleave(n_bits)]

        output_shape  = res_dict['Samples_si'].shape
        output_shape  = torch.Size([output_shape[0],code_inds.max()+1,output_shape[2],output_shape[3],output_shape[4]])

        return locations, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape
# p_col = []

In [ ]:
# #export
# class ISIPostProcess(SIPostProcess):
    
#     def __init__(self, m1_threshold:float = 0.1, samp_threshold=0.1, px_size_zyx=[100,100,100], diag=False):
        
#         super().__init__(m1_threshold = m1_threshold, samp_threshold=samp_threshold, px_size_zyx=px_size_zyx, diag=diag)
#         self.m2_threshold = None
        
#     def forward(self, logits):

#         device = logits.device
#         count = 0
#         p = torch.sigmoid(logits)
        
#         with torch.no_grad():
            
#             p_ret = 0
#             tot_mask = torch.ones_like(p)
# #             count_arr = torch.zeros_like(p)
            
#             while True:
                
#                 count += 1

#                 # probability values > threshold are regarded as possible locations
#                 p_clip = torch.where(p>self.m1_threshold,p,torch.zeros_like(p))*tot_mask

#                 # localize maximum values within a 3x3 patch
#                 pool = F.max_pool3d(p_clip,3,1,padding=1)
#                 max_mask1 = torch.eq(p, pool).float()
#                 max_mask1[p==0] = 0
                
# #                 count_arr += max_mask1*count
                
#                 tot_mask *= (torch.ones_like(max_mask1) - max_mask1)
                
#                 # Add probability values from the adjacent pixels
#                 conv = F.conv3d(p, self.filt.to(device).expand(-1,p.shape[1],-1,-1,-1) ,padding=1)
#                 p_ps = max_mask1 * conv
                
#                 p_ret += torch.clamp_max(p_ps, 1) 
                
#                 p_fac = 1/p_ps
#                 p_fac[torch.isinf(p_fac)] = 0
#                 p_fac = torch.clamp_max(p_fac, 1) 
#                 p_proc = F.conv3d(p_fac, self.filt.to(device),padding=1)*p

#                 p = p - p_proc
#                 torch.clamp_min_(p, 0)
                
#                 if not max_mask1.sum():
#                     break
            
#             return p_ret #, count_arr
        
# #                 plt.figure(figsize=(20,5))
# #                 plt.subplot(141)
# #                 plt.imshow(cpu(p[0,0][sl[1:]]).sum(0))
# #                 plt.subplot(142)
# #                 plt.imshow(cpu(p_ps[0,0][sl[1:]]).sum(0))
# #                 plt.title(cpu(p_ps[0,0][sl[1:]]).sum())
# #                 plt.colorbar()
# #                 plt.subplot(143)
# #                 plt.imshow(cpu(p_proc[0,0][sl[1:]]).sum(0))
# #                 plt.title(cpu(p_proc[0,0][sl[1:]]).sum())
# #                 plt.colorbar()
# #                 plt.show()

In [ ]:
#export
class ISIPostProcess(SIPostProcess):
    
    def __init__(self, m1_threshold:float = 0.1, samp_threshold=0.1, px_size_zyx=[100,100,100], diag=False):
        
        super().__init__(m1_threshold = m1_threshold, samp_threshold=samp_threshold, px_size_zyx=px_size_zyx, diag=diag)
        self.m2_threshold = None
        
    def forward(self, logits):

        device = logits.device
        p = torch.sigmoid(logits)
        
        batch_size = p.shape[0]
        n_codes = p.shape[1]
        
        p = p.reshape(batch_size*n_codes,1,*p.shape[-3:])
        
        with torch.no_grad():
            
            p_SI = 0
            tot_mask = torch.ones_like(p)
            max_mask = torch.ones_like(p)
            
            while max_mask.sum():
                
                # voxels with probability values > threshold,
                # and which where not previously counted as locations, are canditates
                p_cand = torch.where(p>self.m1_threshold, p, torch.zeros_like(p)) * tot_mask

                # localize maximum (nonzero) values within a 3x3x3 volume
                p_cand = F.max_pool3d(p_cand,3,1,padding=1)
                max_mask = torch.eq(p, p_cand).float()
                max_mask[p==0] = 0
                
                # Add up probability values from the adjacent pixels
                conv = F.conv3d(p, self.filt.to(device), padding=1)
                p_sum = max_mask * conv
                
                # Add the integrated probabilities to the return tensor. 
                p_SI += torch.clamp_max(p_sum, 1) 
                # Voxels that where added can not be added again
                tot_mask *= (torch.ones_like(max_mask) - max_mask)
                
                # The probability mass that contributed to p_sum is removed.
                p_fac = 1/p_sum
                p_fac[torch.isinf(p_fac)] = 0
                p_fac = torch.clamp_max(p_fac, 1) 
                p_proc = F.conv3d(p_fac, self.filt.to(device),padding=1)*p

                p = p - p_proc
                torch.clamp_min_(p, 0)
            
            return p_SI.reshape(batch_size,n_codes,*p.shape[-3:])

In [ ]:
from decode_fish.funcs.evaluation import *

post_proc1 = SIPostProcess(m1_threshold=0.03, m2_threshold=0.25, samp_threshold=0.6, px_size_zyx=[100,100,100], diag=True)
post_proc2 = ISIPostProcess(m1_threshold=0.03, samp_threshold=0.5, px_size_zyx=[100,100,100], diag=True)

# matching(px_to_nm(gt_df),  post_proc1.forward(model_out, ret='df'), tolerance=500, print_res=True)
# _=matching(px_to_nm(gt_df),  post_proc2.forward(model_out, ret='df'), tolerance=500, print_res=True)

In [ ]:
from decode_fish.funcs.utils import *
# model_out = torch.load('../data/model_output.pt')
# probs_inp = torch.sigmoid(model_out['logits'])[:,:,:,250:300,200:250]

# model_out = torch.load('../data/model_batch_output.pt')
# probs_inp = torch.sigmoid(model_out['logits'])

# model_out = torch.load('../data/model_output_t.pt')
# probs_inp = torch.sigmoid(model_out['logits'])[:,:,:,:,:]

model_out = torch.load('../data/model_output_2d.pt')
# probs_inp = torch.sigmoid(model_out['logits'])
# model_out = torch.load('../data/model_batch_output_class.pt')
# for m in model_out:
#     model_out[m] =model_out[m].detach() 
# gt_df = torch.load('../data/gt_1.pt')
# len(gt_df)

In [ ]:
model_out['logits'] = model_2d['logits'].expand(-1,140,-1,-1,-1)

NameError: name 'model_2d' is not defined

In [ ]:
tra1 = post_proc2(model_out['logits'])

In [ ]:
tra2 = post_proc2(model_out['logits'])

In [ ]:
plt.imshow(cpu(tra2)[0,0,0])
plt.colorbar()

In [ ]:
post_proc2.get_df(model_out)

In [ ]:
# probs_inp = probs_inp[:,:,10:11]
# for k in model_out:
#     model_out[k] = model_out[k][:,:,10:11]

In [ ]:
# print(len(gt_df))
plt.figure(figsize=(20,10))
plt.subplot(231)
probs = cpu(probs_inp[0,0])
probsf = probs + 0
probsf[probsf<0.01] = 0
im = plt.imshow(probs.sum(0))
# plt.scatter(gt_df['x'],gt_df['y'], color='red', s=5.)
plt.title(f'Net output {probs.sum().item():.1f} and {probsf.sum().item():.1f}'.format())
add_colorbar(im)

recs = post_proc1.get_si_resdict(model_out)
plt.subplot(232)
im = plt.imshow(cpu(recs['Probs_si'][0,0]).max(0))
add_colorbar(im)
N = cpu(recs['Probs_si'][0,0]).sum().item()
plt.title(f'SI Probs SI {N:.1f}')

plt.subplot(235)
im = plt.imshow(cpu(recs['Samples_si'][0,0]).sum(0))
add_colorbar(im)
plt.title(cpu(recs['Samples_si'][0,0]).sum().item())

recs = post_proc2.get_si_resdict(model_out)
plt.subplot(233)
im = plt.imshow(cpu(recs['Probs_si'][0,0]).max(0))
add_colorbar(im)
N = cpu(recs['Probs_si'][0,0]).sum().item()
plt.title(f'ISI Probs SI {N:.1f}')

plt.subplot(236)
im = plt.imshow(cpu(recs['Samples_si'][0,0]).sum(0))
add_colorbar(im)
plt.title(cpu(recs['Samples_si'][0,0]).sum().item())

In [ ]:
sl = np.s_[:,:10,35:45,20:30]
gt_sub = crop_df(gt_df, sl)
p_sub = crop_df(nm_to_px(post_proc2.forward(model_out, ret='df')), sl)
axes=plot_3d_projections(probs[sl[1:]], 'max', size=15)
# print(probs[sl[1:]].sum(), len(gt_sub), len(p_sub))
# axes[0].scatter(gt_sub['x'],gt_sub['y'], color='red', s=5.)
# axes[1].scatter(gt_sub['x'],gt_sub['z'], color='red', s=5.)
# axes[2].scatter(gt_sub['y'],gt_sub['z'], color='red', s=5.)

axes[0].scatter(p_sub['x'],p_sub['y'], color='red', s=15.)
axes[1].scatter(p_sub['x'],p_sub['z'], color='red', s=15.)
axes[2].scatter(p_sub['y'],p_sub['z'], color='red', s=15.)

In [ ]:
p.shape

In [ ]:
for p in p_col:
    plt.imshow(cpu(p[0,0][sl[1:]]).max(0))
#     plt.title(cpu(p[0,0][sl[1:]]).sum())
    plt.colorbar()
    plt.show()

In [ ]:
plt.imshow(cpu(p[0,0][sl[1:]]).max(0))

In [ ]:
plt.hist(probs.reshape(-1).numpy())

In [ ]:
probs_si = post_proc.spatial_integration(probs_inp)

plt.figure(figsize=(10,5))
plt.subplot(121)
probs = probs_inp[0,0].detach().cpu()
# probs[probs<0.01] = 0
im = plt.imshow(probs.max(dim=0).values)
plt.title(probs.sum().item())
add_colorbar(im)
plt.subplot(122)
im = plt.imshow(probs_si[0,0].cpu().max(dim=0).values, vmax=1)
add_colorbar(im)
plt.title(probs_si[0].sum().item())

In [ ]:
model_out = torch.load('../data/model_output_1.pt')
out_df = post_proc2(model_out)
out_df

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(121)
im = plt.imshow(probs_inp[0,0].cpu().max(dim=0).values)
add_colorbar(im)
plt.title(len(out_df))
plt.scatter(out_df['x']/100,out_df['y']/100, color='red', s=5.)

In [ ]:
model_out = torch.load('../data/model_batch_output.pt')

In [ ]:
from decode_fish.engine.psf import LinearInterpolatedPSF
from decode_fish.engine.noise import sCMOS
from decode_fish.engine.point_process import PointProcessUniform
from decode_fish.funcs.plotting import plot_3d_projections
from decode_fish.engine.microscope import Microscope

psf_state = torch.load('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/fishcod/simfish_psf.pkl')
_,xs,ys,zs = psf_state['psf_volume'].shape
psf = LinearInterpolatedPSF(fs_x=xs, fs_y=ys, fs_z=zs, upsample_factor= 1)
psf.load_state_dict(psf_state)

noise = sCMOS()

micro = Microscope(parametric_psf=[psf], noise=noise, multipl=10000).cuda()

point_process = PointProcessUniform(local_rate = torch.ones([1,1,48,48,48]).cuda()*.0001, min_int = 0.5)
locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape = point_process.sample()

In [ ]:
xsim = micro(locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape)
xrec = micro(locs_mod, x_os_mod, y_os_mod, z_os_mod, ints_mod, output_shape_mod)

In [ ]:
plot_3d_projections(xsim[0,0])

In [ ]:
plot_3d_projections(xrec[0,0])

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 20_MERFISH_visualization.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted index.ipynb.
